In [1]:
# Generic imports
import copy
import os
import scipy.interpolate
from mpl_toolkits.mplot3d import Axes3D
from numpy import \
matrix, linspace, meshgrid, sin, cos, array, sqrt, pi, diff, mean, median, std, arcsin, zeros, random, \
size, reshape, shape, vstack, eye, diag, nan_to_num, ones, loadtxt, savetxt, append, squeeze
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from PIL import Image, ImageFont, ImageDraw
import f90nml

# Application-specific imports
import facetbrightnessstuff9 as fbs
import imagestuff as ims
import gradstuff as gds

# Graphics imports and parameters
#%matplotlib inline
%matplotlib notebook
fontsize = 20
linewidth = 2
matplotlib.rcParams.update({'font.size': fontsize})

CUDA not available.


In [2]:
# Parameters
ABCDangle_deg = 15.0
theta = ABCDangle_deg*pi/180

fontsize = 20
linewidth = 2
matplotlib.rcParams.update({'font.size': fontsize})
fnt = ImageFont.load_default()

In [3]:
###### Specify the input data

# Assume all the data are in the same folder
crystalfolder = 'crystals'

# Each block specifies a calibration crystal
#f1 = '2016-08-09_ice1'
#case = 'calibration'
#imageroot = '/2016-08-09_ice1_full2'
#Boxesnamelist = 'Boxes.nml'

# f1 = '2017-06-19_ice1'
# case = 'calibration'
# imageroot = '/figure'
# Boxesnamelist = 'Boxes.nml'

# f1 = '2017-06-26_ice4'
# case = 'calibration'
# imageroot = '/figure'
# Boxesnamelist = 'Boxes.nml'

f1 = '2018-01-11_ice1'
case = 'calibration'
imageroot = '/2018-01-11-full'
Boxesnamelist = 'Boxes.nml'


In [4]:
# These are derivative names
Calibrationfile = crystalfolder+'/'+f1+'/Calibration.nml'
surfaceroot = crystalfolder+'/'+f1+'/'+case
Boxesfile = surfaceroot+'/'+Boxesnamelist

In [5]:
# Just echo input data files 
%type $Boxesfile

ERROR:root:Line magic function `%type` not found.


In [6]:
# Define boxes for calibration
Boxes=f90nml.read(Boxesfile) #reads the file at a given path
nx1list=Boxes['Boxes']['nx1list']
ny1list=Boxes['Boxes']['ny1list']
labellist=Boxes['Boxes']['labellist']
boxsize=Boxes['Boxes']['boxsize']; print (boxsize)

# Packaging these values for subsequent use
nboxes = len(nx1list); print ("nboxes =", nboxes)
nx2list = np.array(nx1list)+boxsize; ny2list = np.array(ny1list)+boxsize

40
nboxes = 3


In [7]:
# Define vectors for calibration
Vectors=f90nml.read(Boxesfile) #reads the file at a given path
xorigin=Vectors['Vectors']['xorigin']
yorigin=Vectors['Vectors']['yorigin']
xa=Vectors['Vectors']['xa']
ya=Vectors['Vectors']['ya']
xb=Vectors['Vectors']['xb']
yb=Vectors['Vectors']['yb']
xc=Vectors['Vectors']['xc']
yc=Vectors['Vectors']['yc']

# Packaging these values for subsequent use
linea = [xorigin,yorigin,xa,ya]
lineb = [xorigin,yorigin,xb,yb]
linec = [xorigin,yorigin,xc,yc]

In [8]:
# Read the input dataset, including spacing in micrometers
dx,dy,cA,cB,cC,cD,Filename = ims.getc2tif(surfaceroot, '/SEMimages', imageroot)
print("Filename, dx and dy", Filename, dx, dy)

Filename, dx and dy crystals/2018-01-11_ice1/calibration/SEMimages/2018-01-11-full-D.tif 0.9921875 0.9921875


<IPython.core.display.Javascript object>


In [9]:
# Graph the segments
im = Image.open(Filename)
ny_im,nx_im = np.shape(im)
draw = ImageDraw.Draw(im)
draw.line(linea, fill=50,width=2)
draw.line(lineb, fill=40,width=2)
draw.line(linec, fill=0,width=2)
draw.text((xa-20,ya-20), 'a', font=fnt)
draw.text((xb-20,yb+20), 'b', font=fnt)   

fig, ax = plt.subplots()
for i in range(nboxes):
    nx1 = nx1list[i]
    nx2 = nx2list[i]
    ny1 = ny1list[i]
    ny2 = ny2list[i]
    ims.myrectanglelabel(draw,(nx1,ny1),(nx2,ny2),labellist[i])
ax.set_title(f1,fontsize=fontsize*0.9)
Iwantmicrons = False
if Iwantmicrons:
    ax.imshow(im,cmap = 'Greys_r', vmin = 0,vmax = 255, extent=[0,nx_im*dx,0,ny_im*dy])
    ax.set_xlabel(r'$x \ (\mu m)$',fontsize=fontsize*1.1)
    ax.set_ylabel(r'$y \ (\mu m)$',fontsize=fontsize*1.1)
else:
    ax.imshow(im,cmap = 'Greys_r', vmin = 0,vmax = 255)


In [10]:
# This is for 2018-01-11_ice1

# Calculate normal vectors of the crystal
avec, bvec, ncvec, navec, nbvec = fbs.solveforabc(\
                 xa-xorigin,ya-yorigin,\
                 xb-xorigin,yb-yorigin,\
                 xc-xorigin,yc-yorigin)
print('unit normal cvec is: \n', ncvec)
print ('unit normal a-prismatic facet =\n',navec)
print ('unit normal b-prismatic facet =\n',nbvec)

Rot28a = ims.myrotation_matrix(avec, -28.)
napvec = Rot28a*navec

Rot28b = ims.myrotation_matrix(bvec, 28.)
nbpvec = Rot28b*nbvec

print('Unit normal a-box (a prismatic) = \n', navec)
print('Unit normal b-box (a pyramidal) = \n', napvec)
print('Unit normal d-box (b pyramidal) = \n', nbpvec)

#make assignments for following code
nbvec = napvec
ndvec = nbpvec



found 16 solutions
physically reasonable solution is # 10
unit normal cvec is: 
 [[ 0.        ]
 [-0.96961609]
 [-0.24463166]]
unit normal a-prismatic facet =
 [[ 0.33836382]
 [-0.23020215]
 [ 0.91242364]]
unit normal b-prismatic facet =
 [[-0.64576131]
 [-0.18678589]
 [ 0.74034003]]
Unit normal a-box (a prismatic) = 
 [[ 0.33836382]
 [-0.23020215]
 [ 0.91242364]]
Unit normal b-box (a pyramidal) = 
 [[ 0.29875752]
 [ 0.25195075]
 [ 0.92046986]]
Unit normal d-box (b pyramidal) = 
 [[-0.57017339]
 [ 0.29028502]
 [ 0.76852905]]


In [11]:
# def getc2(folder,namebase,imageroot): #applies getval to all four images in a set
#     detectors = 'A', 'B', 'C', 'D'
#     for det in detectors:
#         Filename = folder+namebase+imageroot + '-' + det + '.bmp'
#         #print(Filename)
#         value, Nx, Ny, Filename = getval2(Filename)
#         if det == 'A':
#             cA = value #raw bmp data
            
#         if det == 'B':
#             cB = value
            
#         if det == 'C':
#             cC = value
                
#         if det == 'D':
#             cD = value   

#     #Getting pixel size
#     textfile = folder+namebase+imageroot+'-A'+'.txt'
#     fileref = open(textfile,"r")
#     dummy = fileref.read().split()
#     for i in range(len(dummy)):
#         test = dummy[i].find('PixelSize')
#         #print (i, test)
#         if test!=-1:
#             pixelsize = float(dummy[i][10:])/1000.
#             break
#     #pixelsize = float(dummy[16][10:])/1000 #microns
#     fileref.close()
#     dx = dy = pixelsize
    
#     return dx,dy,cA,cB,cC,cD,Filename


In [12]:
# For each detector, get s-values
sAa, sBa, sCa, sDa = ims.mygets(navec,theta)
sAb, sBb, sCb, sDb = ims.mygets(nbvec,theta)
sAd, sBd, sCd, sDd = ims.mygets(ndvec,theta)
slistA = np.squeeze(np.array([sAa, sAb, sAd]))
slistB = np.squeeze(np.array([sBa, sBb, sBd]))
slistC = np.squeeze(np.array([sCa, sCb, sCd]))
slistD = np.squeeze(np.array([sDa, sDb, sDd]))

# For each detector, get s-values
sAa, sBa, sCa, sDa = ims.mygets(navec,theta)
sAb, sBb, sCb, sDb = ims.mygets(nbvec,theta)
sAd, sBd, sCd, sDd = ims.mygets(ndvec,theta)
slistA = np.squeeze(np.array([sAa, sAb, sAd]))
slistB = np.squeeze(np.array([sBa, sBb, sBd]))
slistC = np.squeeze(np.array([sCa, sCb, sCd]))
slistD = np.squeeze(np.array([sDa, sDb, sDd]))

# Extract the observed intensities
cA_obs = []
cB_obs = []
cC_obs = []
cD_obs = []
for isegment in range(nboxes):
    nx1=nx1list[isegment]; nx2=nx2list[isegment] 
    ny1=ny1list[isegment]; ny2=ny2list[isegment]
    cA_obs.append(np.mean(cA[ny1:ny2,nx1:nx2].astype('float')))
    cB_obs.append(np.mean(cB[ny1:ny2,nx1:nx2].astype('float')))
    cC_obs.append(np.mean(cC[ny1:ny2,nx1:nx2].astype('float')))
    cD_obs.append(np.mean(cD[ny1:ny2,nx1:nx2].astype('float')))


<IPython.core.display.Javascript object>


pA = 234.05627828 , 95.4262985217
pB = 233.179138218 , 52.9919877519
pC = 134.103411279 , 65.614688423
pD = 128.145154481 , 77.1796092734


In [13]:
# See what these look like graphically
plt.figure()
markersize = 10
plt.plot(slistA,cA_obs,'or',markersize=15)
plt.plot(slistB,cB_obs,'<b',markersize=15)
plt.plot(slistC,cC_obs,'sy',markersize=15)
plt.plot(slistD,cD_obs,'>g',markersize=15)
plt.legend(['A', 'B', 'C', 'D'],loc='upper left')
plt.plot(slistA[0],cA_obs[0],'k*')
plt.plot(slistA[1],cA_obs[1],'kx')
plt.plot(slistA[2],cA_obs[2],'k+')
plt.plot(slistB[0],cB_obs[0],'k*')
plt.plot(slistB[1],cB_obs[1],'kx')
plt.plot(slistB[2],cB_obs[2],'k+')
plt.plot(slistC[0],cC_obs[0],'k*')
plt.plot(slistC[1],cC_obs[1],'kx')
plt.plot(slistC[2],cC_obs[2],'k+')
plt.plot(slistD[0],cD_obs[0],'k*')
plt.plot(slistD[1],cD_obs[1],'kx')
plt.plot(slistD[2],cD_obs[2],'k+')
plt.grid()
srange = [-.6,.6]
plt.xlim(srange)
plt.xlabel('$s$',fontsize=fontsize*1.5)
plt.ylabel('$c_{obs}$',fontsize=fontsize*1.5)


# Fitting
maxorder = 1
order = min(len(slistA)-1,maxorder)
pA = np.polyfit(slistA,cA_obs,order); print('pA =', pA[0], ',', pA[1])
pB = np.polyfit(slistB,cB_obs,order); print('pB =', pB[0], ',', pB[1])
pC = np.polyfit(slistC,cC_obs,order); print('pC =', pC[0], ',', pC[1])
pD = np.polyfit(slistD,cD_obs,order); print('pD =', pD[0], ',', pD[1])
s_theory = np.linspace(srange[0],srange[1])
cA_theory = np.polyval(pA,s_theory)
cB_theory = np.polyval(pB,s_theory)
cC_theory = np.polyval(pC,s_theory)
cD_theory = np.polyval(pD,s_theory)
plt.plot(s_theory,cA_theory,'-r',linewidth=linewidth)
plt.plot(s_theory,cB_theory,'--b',linewidth=linewidth)
plt.plot(s_theory,cC_theory,'-.y',linewidth=linewidth)
plt.plot(s_theory,cD_theory,':g',linewidth=linewidth)
plt.title(f1,fontsize=fontsize*0.9)

In [14]:
# Save the calibration file
cfile = open(Calibrationfile,'w')
cfile.write('&Calibration\n')
cfile.write('   '+'pA = '+str(pA[0])+',  '+str(pA[1])+'\n')
cfile.write('   '+'pB = '+str(pB[0])+',  '+str(pB[1])+'\n')
cfile.write('   '+'pC = '+str(pC[0])+',  '+str(pC[1])+'\n')
cfile.write('   '+'pD = '+str(pD[0])+',  '+str(pD[1])+'\n')
cfile.write('/ \n')
cfile.close()
%cat $Calibrationfile

&Calibration
   pA = 234.05627828,  95.4262985217
   pB = 233.179138218,  52.9919877519
   pC = 134.103411279,  65.614688423
   pD = 128.145154481,  77.1796092734
/ 
